#  Calibration Image Master Creation notebook

This notebook calibrates Bias images, Dark images and Flat images and combines them into their respective Master files

Made by: Harlan Shaw <harlan.shaw@ucalgary.ca>

## Required Python packages:
[Astropy](https://www.astropy.org/)

[CCDProc](https://ccdproc.readthedocs.io/en/latest/index.html)

[Astro-SCRAPPY](https://github.com/astropy/astroscrappy)

It's recommended you download and install [Anaconda](https://www.anaconda.com/products/individual#Downloads) as this contains a Python environment and Astropy.

You will need to install CCDProc using the Conda install command: `conda install -c conda-forge ccdproc`

This will also install Astropy and Astro-SCRAPPY if not already installed.

## Citations:
With the projects used above, any use of this notebook requires the citations of the Python packages and papers from which they are derived. 
Their citations are located here:

[Astropy](https://www.astropy.org/acknowledging.html)

[CCDProc](https://ccdproc.readthedocs.io/en/latest/citation.html)

[Astro-SCRAPPY](https://github.com/astropy/astroscrappy)



## How to Use

It's recommended you click "Run All Cells" to run this notebook, but you may optionally run each cell individually.

This notebook requires the location of your raw bias images as either a full location or as a relative path if you have the bias images in the folder that this notebook is located in.

Example:

Full location: C:\path\to\bias

Relative: path\to\bias



# Setup

All variables are set up in this cell. If this cell is not run, the program will not run.

In [1]:
from pathlib import Path
from astropy.nddata import CCDData
from astropy.stats import mad_std
from astropy.units import adu, second
from astropy.wcs import FITSFixedWarning
from ccdproc import ImageFileCollection, combine, subtract_bias, subtract_dark, cosmicray_lacosmic
import matplotlib.pyplot as plt
import numpy as np
import logging
import warnings

warnings.filterwarnings("ignore", category=FITSFixedWarning)


GIGABYTE = 1e9 # 1 billion bytes
MEMORY_LIMIT = 10*GIGABYTE

data_directory = Path(".") # directory that notebooks are in.
DARKS_PATH = data_directory / "Darks_Jun"
BIAS_PATH = data_directory / "Bias"
FLATS_PATH = data_directory / "Flats"
reduced_path = data_directory / "reduced"
reduced_path.mkdir(exist_ok=True)
#useful functions
def inv_average(data):
    return 1/np.average(data)

def inv_median(data):
    return 1/np.median(data)


# Stage 1: Bias

During this stage the master Bias is made, it is integral to the rest of the process and must be done first.

In [ ]:
biases = ImageFileCollection(BIAS_PATH, glob_include="*bias*.fit*")
paths = []
for bias in biases.files:
    bias_path = BIAS_PATH / bias
    paths.append(bias_path)
combined_bias = combine(paths, 
                        method="median", 
                        mem_limit = MEMORY_LIMIT,
                        sigma_clip=True,
                        sigma_clip_low_thresh=5,
                        sigma_clip_high_thresh=5,
                        sigma_clip_func=np.median,
                        sigma_clip_dev_func=mad_std,
                        unit=adu)
combined_bias.meta["IMAGETYP"] = "Bias"
combined_bias.meta['HISTORY'] = f"Combined {len(biases.files)} images by median"
combined_bias = combined_bias.to_hdu()
combined_bias[0].scale("int32")
combined_bias[0].writeto(reduced_path / "combined_bias_median.fit")


# Stage 2: Dark

Here the master Dark is made. This process requires the use of the master Bias, so Stage 1 must be run first

In [ ]:
darks = ImageFileCollection(DARKS_PATH, glob_include="*.fit*")
logging.info(f"Found {len(darks.files)} dark files")
INTERMEDIARY_PATH = Path(DARKS_PATH / "calibrated")
INTERMEDIARY_PATH.mkdir(exist_ok=True)

combined_bias_path = reduced_path / "combined_bias_mean.fit"
combined_bias = CCDData.read(combined_bias_path, unit=adu)

for dark in darks.files:
    flat_name = Path(dark).stem 
    modified_flat_name = INTERMEDIARY_PATH / f"{flat_name}_b.fit"
    if not modified_flat_name.is_file():
        dark_ccd = CCDData.read(DARKS_PATH / dark, unit=adu)
        dark_ccd = subtract_bias(ccd=dark_ccd, 
                                master=combined_bias, 
                                add_keyword={"HISTORY": f"Subtracted master bias combined_bias.fit"})
        ccd_exposure = float(dark_ccd.header["exptime"])
        dark_ccd = dark_ccd.to_hdu(hdu_uncertainty=None)
        dark_ccd[0].writeto(modified_flat_name)
paths = []
calib_darks = ImageFileCollection(INTERMEDIARY_PATH, glob_include="*.fit*")
for dark in calib_darks.files:
    dark_path = INTERMEDIARY_PATH / dark
    paths.append(dark_path)
combined_dark = combine(paths, 
                        method="average", 
                        mem_limit = MEMORY_LIMIT,
                        unit=adu)

combined_dark = subtract_bias(ccd=combined_dark, master=combined_bias, add_keyword={"HISTORY": f"Subtracted master bias {combined_bias_path.name}"})
temp = round(combined_dark.header["CCD-TEMP"])
combined_dark.meta["IMAGETYP"] = "Dark"
combined_dark.meta['combined'] = True
combined_dark.meta['HISTORY'] = f"Combined {len(darks.files)} images by mean"
combined_dark = combined_dark.to_hdu(hdu_uncertainty=None)
combined_dark[0].scale("int32")
combined_dark[0].writeto(reduced_path / f"combined_dark_{temp}C_mean.fit")


# Stage 3: Flat

Here the master Flat is made. This process requires the use of the master Bias and master Dark, so Stage 1 and 2 must be run first

### Part 1: Calibrate the flat files

In [40]:
flats = ImageFileCollection(FLATS_PATH, glob_include="*")
INTERMEDIARY_PATH = Path(FLATS_PATH / "calibrated")
INTERMEDIARY_PATH.mkdir(exist_ok=True)
combined_bias = CCDData.read(reduced_path /"combined_bias_mean.fit")
combined_dark = CCDData.read(reduced_path /"combined_dark_10C_mean.fit")

for flat in flats.files:
    flat_name = Path(flat).stem 
    modified_flat_name = INTERMEDIARY_PATH / f"{flat_name}_b_d.fit"
    if not modified_flat_name.is_file():
        flat_ccd = CCDData.read(FLATS_PATH / flat, unit=adu)
        flat_ccd = subtract_bias(ccd=flat_ccd, 
                                master=combined_bias, 
                                add_keyword={"HISTORY": f"Subtracted master bias combined_bias.fit"})
        flat_ccd = subtract_dark(ccd=flat_ccd, 
                                master=combined_dark, 
                                exposure_time="EXPTIME", 
                                exposure_unit=second, 
                                scale=True,
                                add_keyword={"HISTORY": f"Subtracted master bias combined_dark.fit"})
        #flat_ccd = cosmicray_lacosmic(ccd=flat_ccd, gain=1.42, readnoise=15.1, fsmode="convolve", cleantype="median", gain_apply=True)
        ccd_exposure = float(flat_ccd.header["exptime"])
        flat_ccd = flat_ccd.to_hdu(hdu_uncertainty=None)
        flat_ccd[0].writeto(modified_flat_name)

### Part 2: Combine the flat files

In [3]:

INTERMEDIARY_PATH = Path(FLATS_PATH / "calibrated") 
calibrated_flats = ImageFileCollection(INTERMEDIARY_PATH, glob_include="*.fit*")
paths = []
for flat in calibrated_flats.files:
    flat_path = INTERMEDIARY_PATH / flat
    paths.append(flat_path)
    
combined_flat = combine(paths, 
                        method="median", 
                        mem_limit = MEMORY_LIMIT,
                        scale= inv_median,
                        sigma_clip=True,
                        sigma_clip_low_thresh=5,
                        sigma_clip_high_thresh=5,
                        sigma_clip_func=np.median,
                        sigma_clip_dev_func=mad_std,
                        dtype="float32",
                        unit=adu)
combined_flat.meta["IMAGETYP"] = "Flat"
combined_flat.meta['combined'] = True
combined_flat.meta['HISTORY'] = f"Combined {len(paths)} images by median"

combined_flat = combined_flat.to_hdu(hdu_uncertainty=None)
combined_flat[0].writeto(reduced_path / "combined_flat_mean.fit")

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:splitting each image into 3 chunks to limit memory usage to 10000000000.0 bytes.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: splitting each image into 3 chunks to limit memory usage to 10000000000.0 bytes. [ccdproc.combiner]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


...

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


In [ ]:
import matplotlib.pyplot as plt
from astropy.units import electron
from astropy.visualization import hist
cool_dark = CCDData.read(reduced_path / "combined_dark_1c_mean.fit", unit=adu).multiply(1.42 * electron / adu).divide(90*second)
old_dark = CCDData.read(reduced_path / "combined_dark_10c_mean.fit", unit=adu).multiply(1.42 * electron / adu).divide(60*second)

plt.figure(figsize=(20, 10))

hist(cool_dark.data.flatten(), bins=5000, density=False, label='90 sec cool dark', alpha=0.4)
hist(old_dark.data.flatten(), bins=5000, density=False, label='60 sec old dark', alpha=0.4)
plt.xlabel('dark current, $e^-$/sec')
plt.ylabel('Number of pixels')
plt.loglog()
plt.legend();

In [ ]:
hot_pixels = (cool_dark.data > 3.0)
plt.figure(figsize=(10, 10))
plt.plot(cool_dark.data[hot_pixels].flatten(), old_dark.data[hot_pixels].flatten(), '.', alpha=0.2, label='Data')
plt.xlabel("dark current ($e^-$/sec), 90 sec exposure time")
plt.ylabel("dark current ($e^-$/sec), 60 sec exposure time")
plt.xlim((1, 100))
plt.ylim((1, 100))
plt.plot([0, 100], [0, 100], label='Ideal relationship')
plt.grid()
plt.legend();

In [ ]:
from astropy.units import dimensionless_unscaled
hot_pixels.sum()
as_ccd = CCDData(data=hot_pixels.astype("uint8"), unit=dimensionless_unscaled)
as_ccd.header["imagetyp"] = "dark mask"
as_ccd.write(reduced_path / "dark_current_mask.fit")

In [ ]:
from ccdproc import ccdmask

flat = reduced_path / "combined_flat.fit"
flat_ccd = CCDData.read(flat)

flatmask = ccdmask(flat_ccd)
mask_as_ccd = CCDData(data=flatmask.astype("uint8"), unit=dimensionless_unscaled)
mask_as_ccd.header["imagetyp"] = "flat mask"
mask_as_ccd.write(reduced_path / "flat_mask.fit")


In [ ]:
combined_mask = mask_as_ccd.data | as_ccd.data
combined_mask = CCDData(data=combined_mask.astype("uint8"), unit=dimensionless_unscaled)
combined_mask.write(reduced_path / "combined_mask.fit")

In [ ]:
combined_mask.data.sum()